<a href="https://colab.research.google.com/github/rajeshsahu09/CS69002_9A_18CS60R19/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

### Import Header Files

In [0]:
import torch
import pandas as pd
import numpy as np
import io

### Load the Dataset and Visualise

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
# df_train.head()
url = "https://raw.githubusercontent.com/rajeshsahu09/CS69002_9A_18CS60R19/master/Dataset/Train_20K.csv?token=As1EmIQyHMN7oJnDLYe-xuY8qLC9Ngoxks5cn1RdwA%3D%3D"
df_train = pd.read_csv(url, sep='\t')

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_val = pd.read_csv(io.StringIO(uploaded['check.csv'].decode('utf-8')), sep='\t')
# df_val.head()
url = "https://raw.githubusercontent.com/rajeshsahu09/CS69002_9A_18CS60R19/master/Dataset/check.csv?token=As1EmF_ArLoFbcmGB3h1CAXtXcr8h2nPks5cn1TGwA%3D%3D"
df_val = pd.read_csv(url, sep='\t')

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.csv'].decode('utf-8')), sep='\t')
# df_test.head()
url = "https://raw.githubusercontent.com/rajeshsahu09/CS69002_9A_18CS60R19/master/Dataset/Test_5K.csv?token=As1EmMtuIGJiHwMLRRh0GThgeMlnS1TUks5cn1UPwA%3D%3D"
df_test = pd.read_csv(url, sep='\t')

In [8]:
print('Number of Negative movie reviews', len(df_train[df_train['label']==0]))
print('Number of Positive movie reviews', len(df_train[df_train['label']==1]))
print('Number of movie reviews', len(df_train['label']))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


### Data pre-processing

In [0]:
raw_text_reviews = df_train['text'].astype(str).tolist()
len(raw_text_reviews), raw_text_reviews[-2]

(17999,
 'Anyone notice that Tommy only has 3 facial expressions.<br /><br />1. The angry eyes look he gives every enemy. 2. The holding of the hands to face, mouth agape and frightened eyes. 3. The smiling Tommy Turnbull.<br /><br />I have to say that i pretty much hate this show, i don\'t watch it but it\'s like Code Lyoko, we\'ve all watched at least one, i must say that this show is borderline racist, uninteresting and pointless.<br /><br />every episode ends with robotboy winning, except for one exception when robotboy basically let this overly geeky freakazoid fly away on a jetpack.<br /><br />The jokes are pretty crude too, i think it\'s mostly people saying the word "Suck" or farting, i think the bullies of the show are quite shocking too.<br /><br />Isn\'t there one that hides a bowling ball under his hate, and the other uses a chain, for god sake, what kind of school is he going to. Not to mention his older brother, who is borderline psychopathic and has no other character qu

In [0]:
text_labels = df_train['label'].astype(int)
len(text_labels), text_labels[len(text_labels)-2]

(17999, 0)

In [0]:
import re
import string
def preprocess_document(doc):
  # negative sense should not be eleminated + some short representation
  CONTRACTIONS = {"mayn't":"may not", "can't":"can not", "won't":"will not", "isn't":"is not", "amn't":"am not",\
                  "aren't":"are not", "wasn't":"was not", "weren't":"were not", "couldn't":"could not", \
                  "wouldn't":"would not", "don't":"do not", "doesn't":"does not",\
                  "i'll":"i will", "you'll":"you will", "they'll":"they have",\
                  "may've":"may have", "can've":"can have", "will've":"will have", "you've":"you have", \
                  "could've":"could have", "would've":"would have", "they":"they have",\
                  "i've":"i have", "you've":"you have", "we've":"we have", "there's":"there is", "i'am":"i am", "it's":"it is"}
  punctuation = string.punctuation + "\n\n"
  punc_replace = ''.join([' ' for s in punctuation]); # required for replacing punctuation with null ('')
  doc_clean = doc.replace('-', ' '); # replace - with null str
  doc_clean = doc_clean.replace('<br />', ''); # replace <br /> with ''
  doc_clean = doc_clean.replace("’", "'"); # replace <br /> with null str
  doc_clean = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in doc_clean.split(' ')] # replacing some common short forms
  doc_clean = " ".join(doc_clean) # list to sentence of strings
  doc_clean = re.sub(r'\W +', ' ', doc_clean) # except [a-zA-Z0-9_]
  doc_clean = re.sub(r'\d+', ' ', doc_clean) # remove numbers [0-9]
  trans_table = str.maketrans(punctuation, punc_replace); # replace punctuations with ' '
  doc_clean = ' '.join([word.translate(trans_table) for word in doc_clean.split(' ')]);
  doc_clean = doc_clean.split(' ');
  doc_clean = [word for word in doc_clean if len(word) > 0];

  return doc_clean;

In [0]:
train_text_reviews = [preprocess_document(review.lower()) for review in raw_text_reviews]
print (train_text_reviews[len(train_text_reviews)-2])
print (text_labels[len(text_labels)-2])

['anyone', 'notice', 'that', 'tommy', 'only', 'has', 'facial', 'expressions', 'the', 'angry', 'eyes', 'look', 'he', 'gives', 'every', 'enemy', 'the', 'holding', 'of', 'the', 'hands', 'to', 'face', 'mouth', 'agape', 'and', 'frightened', 'eyes', 'the', 'smiling', 'tommy', 'turnbull', 'i', 'have', 'to', 'say', 'that', 'i', 'pretty', 'much', 'hate', 'this', 'show', 'i', 'do', 'not', 'watch', 'it', 'but', 'it', 'is', 'like', 'code', 'lyoko', 'we', 'have', 'all', 'watched', 'at', 'least', 'one', 'i', 'must', 'say', 'that', 'this', 'show', 'is', 'borderline', 'racist', 'uninteresting', 'and', 'pointless', 'every', 'episode', 'ends', 'with', 'robotboy', 'winning', 'except', 'for', 'one', 'exception', 'when', 'robotboy', 'basically', 'let', 'this', 'overly', 'geeky', 'freakazoid', 'fly', 'away', 'on', 'a', 'jetpack', 'the', 'jokes', 'are', 'pretty', 'crude', 'too', 'i', 'think', 'it', 'is', 'mostly', 'people', 'saying', 'the', 'word', 'suck', 'or', 'farting', 'i', 'think', 'the', 'bullies', 'of

###Creating Bag Of Word (BOW) representation of sentences.

In [0]:
# Fetching the tokens (all words in a doc) and types(unique words in a doc) in the document
types = ()
tokens = []
for row in train_text_reviews:
    tokens += [words for words in row]
types = set(tokens)

VOCAB_SIZE, TOKEN_SIZE = len(types), len(tokens)
VOCAB_SIZE, TOKEN_SIZE

(65624, 4276056)

In [0]:
char_to_ix = {x:i for i, x in enumerate(types)}
ix_to_char = [x for i, x in enumerate(types)]

ix_to_char[char_to_ix['kick']], char_to_ix['kick'], len(ix_to_char)

('kick', 57477, 65624)

In [0]:
for i, (key, value) in enumerate(char_to_ix.items()):
  if(i > 10):
    break
  print (key,':', value)

dogmas : 0
traders : 1
crucification : 2
rebuilding : 3
playboys : 4
advised : 5
enshrine : 6
settlefor : 7
quarry : 8
workers : 9
lisping : 10


## Intro to PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable # for automatically implement backprop

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

lin = nn.Linear(5,3)
data1 = Variable(torch.randn(10,5)) # create a 10 * 5 matrix of random vars